# Tabella 3

In [1]:
from helper_rasar_datafusion import *

db_mortality, db_datafusion = load_data_rasar_datafusion('lc_db_processed.csv', 'datafusion_db_processed.csv')
print(ctime())
db_label = create_label_rasar('lc_db_processed.csv', 'datafusion_db_processed.csv')
cv_datafusion_rasar(db_mortality, db_datafusion, db_label)

Tue Feb 23 12:10:17 2021
Tue Feb 23 12:25:07 2021
Tue Feb 23 12:35:00 2021
Tue Feb 23 12:44:44 2021
Tue Feb 23 12:54:40 2021
Tue Feb 23 13:04:43 2021
Tue Feb 23 13:14:34 2021
Accuracy:    0.9205585310389302 se: 0.0011510478903421582
Sensitivity: 0.8779131947364638 se: 0.0028392185365880827
Specificity: 0.9471088518614348 se: 0.0009868549679797767
F1 score:    0.8943372987306587 se: 0.0012875171042234885


# Tabella 1

In [1]:
from helper_rasar_datafusion import *

db = pd.read_csv('lc_db_processed.csv').drop(columns = 'Unnamed: 0')
db_df = pd.read_csv('datafusion_db_processed.csv').drop(columns = 'Unnamed: 0')

db = pd.concat([db,
                pd.DataFrame(pd.DataFrame(db['pubchem2d'].values).\
                             apply(lambda x: x.str.replace('', ' ').str.strip().str.split(' '), 
                                                                        axis = 1)[0].to_list(),
                   columns = ['pub'+ str(i) for i in range(1,882)])],
               axis = 1)

db_df = pd.concat([db_df,
                    pd.DataFrame(pd.DataFrame(db_df['pubchem2d'].values).\
                                 apply(lambda x: x.str.replace('', ' ').str.strip().str.split(' '), 
                                                                            axis = 1)[0].to_list(),
                       columns = ['pub'+ str(i) for i in range(1,882)])],
                   axis = 1)

numerical = ['atom_number', 'bonds_number','Mol', 'MorganDensity', 'LogP',
            'alone_atom_number', 'doubleBond', 'tripleBond', 'ring_number', 'oh_count', 'MeltingPoint', 'WaterSolubility']

for nc in numerical:
        minmax = MinMaxScaler()
        minmax.fit(db[[nc]])
        db[[nc]] = minmax.transform(db[[nc]])

for nc in numerical:
    minmax = MinMaxScaler()
    minmax.fit(db_df[[nc]])
    db_df[[nc]] = minmax.transform(db_df[[nc]])


categorical = ['obs_duration_mean',
               'conc1_type', 'exposure_type', 'control_type', 'media_type',
               'application_freq_unit', 'species', 'class', 'tax_order', 'family', 'genus']

db.drop(columns = categorical, inplace = True)
db_df.drop(columns = categorical, inplace = True)

db = db.groupby('test_cas').agg('median').reset_index()
db_df = db_df.groupby(['test_cas', 'endpoint', 'effect']).agg('median').reset_index()

db['target'] = np.where(db['conc1_mean']> 1, 0,1)
db_df['target'] = np.where(db_df['conc1_mean']> 1, 0,1)

db.drop(columns = ['conc1_mean'], inplace = True)
db_df.drop(columns = ['conc1_mean'], inplace = True)

In [3]:
def create_label_rasar_tab1(db, df):
    
    comparing = ['test_cas']

    grouped_datafusion = df.groupby(by=['endpoint', 'effect'])

    db_datafusion_rasar_label = pd.DataFrame()

    for g in grouped_datafusion.groups:
        name = g[0] + '_' + g[1] + '_' + 'label'

        group = grouped_datafusion.get_group(g).drop(columns = ['endpoint', 'effect'])
    
        db_datafusion_rasar_label[name] = db.apply(
            lambda x: find_similar_exp(x, group, comparing), axis = 1).reset_index(drop = True)
        
    return db_datafusion_rasar_label    

In [4]:
print(ctime())
db_label = create_label_rasar_tab1(db, db_df)
db.drop(columns = 'test_cas', inplace = True)
db_df.drop(columns = 'test_cas', inplace = True)

cv_datafusion_rasar(db, db_df, db_label)

Tue Feb 23 18:48:03 2021
Tue Feb 23 18:48:53 2021
Tue Feb 23 18:49:01 2021
Tue Feb 23 18:49:09 2021
Tue Feb 23 18:49:17 2021
Tue Feb 23 18:49:24 2021
Tue Feb 23 18:49:32 2021
Accuracy:    0.8299223441706356 se: 0.001961120478081154
Sensitivity: 0.3632890563926483 se: 0.020474687512452773
Specificity: 0.9542000271905302 se: 0.007621162309837269
F1 score:    0.47048136205221597 se: 0.015984854516224244


# Tabella 2 -- Fathead Minnow

In [1]:
from helper_rasar_datafusion import *

db = pd.read_csv('lc_db_processed.csv').drop(columns = 'Unnamed: 0')
db_df = pd.read_csv('datafusion_db_processed.csv').drop(columns = 'Unnamed: 0')

db = pd.concat([db,
                pd.DataFrame(pd.DataFrame(db['pubchem2d'].values).\
                             apply(lambda x: x.str.replace('', ' ').str.strip().str.split(' '), 
                                                                        axis = 1)[0].to_list(),
                   columns = ['pub'+ str(i) for i in range(1,882)])],
               axis = 1)

db_df = pd.concat([db_df,
                    pd.DataFrame(pd.DataFrame(db_df['pubchem2d'].values).\
                                 apply(lambda x: x.str.replace('', ' ').str.strip().str.split(' '), 
                                                                            axis = 1)[0].to_list(),
                       columns = ['pub'+ str(i) for i in range(1,882)])],
                   axis = 1)

numerical = ['atom_number', 'bonds_number','Mol', 'MorganDensity', 'LogP',
            'alone_atom_number', 'doubleBond', 'tripleBond', 'ring_number', 'oh_count', 'MeltingPoint', 'WaterSolubility']

for nc in numerical:
        minmax = MinMaxScaler()
        minmax.fit(db[[nc]])
        db[[nc]] = minmax.transform(db[[nc]])

for nc in numerical:
    minmax = MinMaxScaler()
    minmax.fit(db_df[[nc]])
    db_df[[nc]] = minmax.transform(db_df[[nc]])


species = pd.read_csv('C:/Users/Simone/Desktop/Utilità tesi magistrale/data/species.txt', sep = '\|', engine = 'python')

fm = species[species['common_name'] == 'Fathead Minnow'][
    ['class', 'tax_order', 'family', 'genus', 'species']].values.ravel()

X_train = db.loc[(db[
    ['class', 'tax_order', 'family', 'genus', 'species']] == fm).all(axis =1)].copy().reset_index(drop=True)
y_train = np.where(X_train['conc1_mean'] > 1, 0,1)

X_test = db.loc[~(db[
    ['class', 'tax_order', 'family', 'genus', 'species']] == fm).all(axis = 1)].copy().reset_index(drop=True)
y_test = np.where(X_test['conc1_mean'] > 1, 0,1)

db_df['target'] = np.where(db_df['conc1_mean']> 1, 0,1)

categorical = ['fish', 'smiles', 'pubchem2d', 'conc1_mean', 'obs_duration_mean',
               'conc1_type', 'exposure_type', 'control_type', 'media_type',
               'application_freq_unit', 'species', 'class', 'tax_order', 'family', 'genus']

X_train.drop(columns = categorical, inplace = True)
X_test.drop(columns = categorical, inplace = True)
categorical.remove('fish')
db_df.drop(columns = categorical, inplace = True)

In [2]:
def create_label_rasar_tab1(db, df):
    
    comparing = ['test_cas']

    grouped_datafusion = df.groupby(by=['endpoint', 'effect'])

    db_datafusion_rasar_label = pd.DataFrame()

    for g in grouped_datafusion.groups:
        name = g[0] + '_' + g[1] + '_' + 'label'

        group = grouped_datafusion.get_group(g).drop(columns = ['endpoint', 'effect'])
    
        db_datafusion_rasar_label[name] = db.apply(
            lambda x: find_similar_exp(x, group, comparing), axis = 1).reset_index(drop = True)
        
    return db_datafusion_rasar_label  

def rasar_train_test(db_train, db_test, y_train, y_test, db_datafusion, db_label_train, db_label_test):
    
    db_simple_rasar_train = df_train_simple_rasar(db_train, y_train)
    
    db_simple_rasar_test = df_test_simple_rasar(db_train, db_test, y_train, y_test)
    
    db_datafusion_rasar_train = df_datafusion_rasar(db_datafusion, db_train)
    
    db_datafusion_rasar_test = df_datafusion_rasar(db_datafusion, db_test)
    
    X_train_rasar = pd.concat([db_simple_rasar_train[['dist_neigh0', 'dist_neigh1']], db_datafusion_rasar_train,
                               db_label_train], axis = 1)
    X_test_rasar = pd.concat([db_simple_rasar_test[['dist_neigh0', 'dist_neigh1']], db_datafusion_rasar_test,
                             db_label_test], axis = 1)
    
    return X_train_rasar, X_test_rasar

print('label_train', ctime())
db_label_train = create_label_rasar_tab1(X_train, db_df)
print('label_test', ctime())
db_label_test = create_label_rasar_tab1(X_test, db_df)

X_train.drop(columns = 'test_cas', inplace = True)
X_test.drop(columns = 'test_cas', inplace = True)
db_df.drop(columns = 'test_cas', inplace = True)
print('db_train e test', ctime())
X_train_rasar, X_test_rasar = rasar_train_test(X_train, X_test, y_train, y_test, db_df, db_label_train, db_label_test)
print('fine', ctime())

label_train Wed Feb 24 11:36:08 2021
label_test Wed Feb 24 11:37:14 2021
db_train e test Wed Feb 24 11:43:56 2021
fine Wed Feb 24 11:47:19 2021


In [4]:
clf = RandomForestClassifier(n_jobs = -1)
clf.fit(X_train_rasar, y_train)
y_pred = clf.predict(X_test_rasar)

tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

print(ctime())
print("Accuracy: ", accuracy_score(y_test, y_pred),
"\n Sensitivity:", recall_score(y_test, y_pred),
"\n Specificity:", tn/(tn+fp),
"\n F1 score:", f1_score(y_test, y_pred))

Wed Feb 24 11:55:31 2021
Accuracy:  0.8190542259220719 
 Sensitivity: 0.6608203677510608 
 Specificity: 0.9284317559640204 
 F1 score: 0.7490780824114157


# Rainbow Trout

In [1]:
from helper_rasar_datafusion import *

db = pd.read_csv('lc_db_processed.csv').drop(columns = 'Unnamed: 0')
db_df = pd.read_csv('datafusion_db_processed.csv').drop(columns = 'Unnamed: 0')

db = pd.concat([db,
                pd.DataFrame(pd.DataFrame(db['pubchem2d'].values).\
                             apply(lambda x: x.str.replace('', ' ').str.strip().str.split(' '), 
                                                                        axis = 1)[0].to_list(),
                   columns = ['pub'+ str(i) for i in range(1,882)])],
               axis = 1)

db_df = pd.concat([db_df,
                    pd.DataFrame(pd.DataFrame(db_df['pubchem2d'].values).\
                                 apply(lambda x: x.str.replace('', ' ').str.strip().str.split(' '), 
                                                                            axis = 1)[0].to_list(),
                       columns = ['pub'+ str(i) for i in range(1,882)])],
                   axis = 1)

numerical = ['atom_number', 'bonds_number','Mol', 'MorganDensity', 'LogP',
            'alone_atom_number', 'doubleBond', 'tripleBond', 'ring_number', 'oh_count', 'MeltingPoint', 'WaterSolubility']

for nc in numerical:
        minmax = MinMaxScaler()
        minmax.fit(db[[nc]])
        db[[nc]] = minmax.transform(db[[nc]])

for nc in numerical:
    minmax = MinMaxScaler()
    minmax.fit(db_df[[nc]])
    db_df[[nc]] = minmax.transform(db_df[[nc]])


species = pd.read_csv('C:/Users/Simone/Desktop/Utilità tesi magistrale/data/species.txt', sep = '\|', engine = 'python')

fm = species[species['common_name'] == 'Rainbow Trout'][
    ['class', 'tax_order', 'family', 'genus', 'species']].values.ravel()

X_train = db.loc[(db[
    ['class', 'tax_order', 'family', 'genus', 'species']] == fm).all(axis =1)].copy().reset_index(drop=True)
y_train = np.where(X_train['conc1_mean'] > 1, 0,1)

X_test = db.loc[~(db[
    ['class', 'tax_order', 'family', 'genus', 'species']] == fm).all(axis = 1)].copy().reset_index(drop=True)
y_test = np.where(X_test['conc1_mean'] > 1, 0,1)

db_df['target'] = np.where(db_df['conc1_mean']> 1, 0,1)

categorical = ['fish', 'smiles', 'pubchem2d', 'conc1_mean', 'obs_duration_mean',
               'conc1_type', 'exposure_type', 'control_type', 'media_type',
               'application_freq_unit', 'species', 'class', 'tax_order', 'family', 'genus']

X_train.drop(columns = categorical, inplace = True)
X_test.drop(columns = categorical, inplace = True)
categorical.remove('fish')
db_df.drop(columns = categorical, inplace = True)

def create_label_rasar_tab1(db, df):
    
    comparing = ['test_cas']

    grouped_datafusion = df.groupby(by=['endpoint', 'effect'])

    db_datafusion_rasar_label = pd.DataFrame()

    for g in grouped_datafusion.groups:
        name = g[0] + '_' + g[1] + '_' + 'label'

        group = grouped_datafusion.get_group(g).drop(columns = ['endpoint', 'effect'])
    
        db_datafusion_rasar_label[name] = db.apply(
            lambda x: find_similar_exp(x, group, comparing), axis = 1).reset_index(drop = True)
        
    return db_datafusion_rasar_label  

def rasar_train_test(db_train, db_test, y_train, y_test, db_datafusion, db_label_train, db_label_test):
    
    db_simple_rasar_train = df_train_simple_rasar(db_train, y_train)
    
    db_simple_rasar_test = df_test_simple_rasar(db_train, db_test, y_train, y_test)
    
    db_datafusion_rasar_train = df_datafusion_rasar(db_datafusion, db_train)
    
    db_datafusion_rasar_test = df_datafusion_rasar(db_datafusion, db_test)
    
    X_train_rasar = pd.concat([db_simple_rasar_train[['dist_neigh0', 'dist_neigh1']], db_datafusion_rasar_train,
                               db_label_train], axis = 1)
    X_test_rasar = pd.concat([db_simple_rasar_test[['dist_neigh0', 'dist_neigh1']], db_datafusion_rasar_test,
                             db_label_test], axis = 1)
    
    return X_train_rasar, X_test_rasar

print('label_train', ctime())
db_label_train = create_label_rasar_tab1(X_train, db_df)
print('label_test', ctime())
db_label_test = create_label_rasar_tab1(X_test, db_df)

X_train.drop(columns = 'test_cas', inplace = True)
X_test.drop(columns = 'test_cas', inplace = True)
db_df.drop(columns = 'test_cas', inplace = True)
print('db_train e test', ctime())
X_train_rasar, X_test_rasar = rasar_train_test(X_train, X_test, y_train, y_test, db_df, db_label_train, db_label_test)
print('fine', ctime())


clf = RandomForestClassifier(n_jobs = -1)
clf.fit(X_train_rasar, y_train)
y_pred = clf.predict(X_test_rasar)

tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

print(ctime())
print("Accuracy: ", accuracy_score(y_test, y_pred),
"\n Sensitivity:", recall_score(y_test, y_pred),
"\n Specificity:", tn/(tn+fp),
"\n F1 score:", f1_score(y_test, y_pred))

label_train Wed Feb 24 11:59:27 2021
label_test Wed Feb 24 12:00:25 2021
db_train e test Wed Feb 24 12:08:12 2021
fine Wed Feb 24 12:11:22 2021
Wed Feb 24 12:11:23 2021
Accuracy:  0.8319523269012485 
 Sensitivity: 0.8567348165965123 
 Specificity: 0.8169219547775346 
 F1 score: 0.7937878682359495
